<h1>0. Packages</h1>

In [1]:
from importlib import reload
import numpy as np
import re
import skyfield.sgp4lib as spg4
from   skyfield.api import wgs84
import matplotlib 
import PyQt5

# User defined lib.
import generate_debris as gd
import CoordTransforms as ct
import visualize_deb as vis
import planetary_data

reload(gd)
reload(ct)

matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy.stats import uniform
from scipy.stats import norm
import scipy.stats as stats
from enum import IntEnum    
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from itertools import product, combinations
from matplotlib import cm
from scipy import integrate
import pandas as pd
from numba import njit, prange


%matplotlib qt5

debris_category = IntEnum('Category', 'rb sc soc')

<IPython.core.display.Javascript object>

<h1>1. Data Structure</h1>

<h3>1.1 Satellite Structure</h3>

Creating an object to represent satellite from 3le.txt <br>
May end up using implementation from another package

<b>Data sources:<b><br>
https://www.space-track.org    
https://www.celestrak.com/NORAD/documentation/spacetrk.pdf

<b>Information about 3le/2le:</b><br>
https://en.wikipedia.org/wiki/Two-line_element_set#cite_note-nasahelp-12
https://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/SSOP_Help/tle_def.html

In [ ]:
class two_line_element:
    
    def __init__(self, data):
        self.parse_data(data)
        
    def parse_data(self, satelite_data):
        self.title = satelite_data[0]
        line_one = satelite_data[1].split()
        line_two = satelite_data[2].split()

        self.catalog_number = re.search('[0-9]{1,5}', line_one[0]).group()
        self.classification = re.search('[UCS]', line_one[0]).group()

        international_designator = re.search('([0-9]{2})([0-9]{3})(\w{1,3})', line_one[1])
        self.launch_year = international_designator.group(1)
        self.launch_number = international_designator.group(2)
        self.launch_piece =  international_designator.group(3)

        epoch = re.search('([0-9]{2})(\d+\.\d+)', line_one[2])
        self.epoch_year = epoch.group(1)
        self.epoch_day = epoch.group(2)

        self.ballistic_coefficient = float(line_one[3])
        self.mean_motion_double_prime = line_one[4]
        self.BSTAR = line_one[5]

        self.inclination = line_two[1]
        self.right_ascension_of_ascending_node = line_two[2]
        self.eccentricity = line_two[3]
        self.argument_of_perigee = line_two[4]

        mean_anomaly = line_two[5]

<h3>Data structure implementation using spg4 from skyfield library<h3>

In [3]:
# Opening the .txt file
with open("3le.txt") as f:
    txt = f.read()
    
sat_lines = re.findall('(.*?)\n(.*?)\n(.*?)\n', txt)

# Preview of `sat_lines`, each element should be the three lines representing a satellite
print(sat_lines[0])

# Convert each group of 3 lines into a satelite object
def line_element_to_satellite(lines):
    title = lines[0]
    line1 = lines[1]
    line2 = lines[2]
    return spg4.EarthSatellite(line1, line2, name=title)

satellites = [line_element_to_satellite(lines) for lines in sat_lines]

# Verify data structure by finding element representing the ISS
starlink_index = [sat.name for sat in satellites].index("0 STARLINK-31")
starlink = satellites[starlink_index]
print("STARLINK-1026 TLE:")
print(starlink)

('0 VANGUARD 1', '1     5U 58002B   19352.86539842  .00000184  00000-0  20948-3 0  9994', '2     5  34.2610  67.7117 1845282 255.2293  83.7559 10.84789954185624')
STARLINK-1026 TLE:
0 STARLINK-31 catalog #44235 epoch 2019-12-19 06:00:01 UTC


In [ ]:
np.rad2deg(starlink.model.inclo)

<h3>1.2 Celestial Bodies</h3>


de421.bsp is a Ephemerides provided by JPL Horizon which has the calculated positions of celestial bodies within a certain time interval. de421 is commonly used due to its small size and its relativley up to date information

In [4]:
from skyfield.api import load, EarthSatellite
ts = load.timescale(builtin=True)

# Loading the data from de421.bsp using skyfield 
planets = load('de421.bsp')

# Finding the data about Earth
earth = planets["Earth"]

<h3>1.3 Celestial Bodies</h3>

Plotting the satellite data in a 3D view.<br>
Accidently broke my implementation so need to start with the below base and get it to plot the ISS again.

<b>Source:</b> https://space.stackexchange.com/questions/25958/how-can-i-plot-a-satellites-orbit-in-3d-from-a-tle-using-python-and-skyfield

<h1>2. Breakup Models</h1>

<h3>2.1 NASA breakup model</h3>

<b>Implementation:</b> (Found on page 207)<br>
https://www.researchgate.net/publication/295490674_Space_debris_cloud_evolution_in_Low_Earth_Orbit

<b>Alternate implementation:</b><br>
https://gitlab.obspm.fr/apetit/nasa-breakup-model/tree/master

<b>Information:</b><br>
https://digitalcommons.calpoly.edu/cgi/viewcontent.cgi?referer=https://en.wikipedia.org/&httpsredir=1&article=1094&context=theses


<b>Now implemented in file generate_debris.py</b><br>

<h3>2.2 NASA breakup model validation</h3>

In [ ]:
# Explosion Case

reload(gd)

m_target = 1000 # [kg]

# Do not need to specify projectile mass, and collision velocity for an explosion
L_c, areas, masses, AM = gd.fragmentation(m_target, 0, 0, False, debris_category.rb, True)

bins = [1e-3, 1e-2, 1e-1, 1, np.inf] # L_c bins in meters
h,b = np.histogram(L_c, bins=bins)

ch = [ { f'>{bins[i]}m':np.sum(h[i:]) for i in range(len(h))}]

df = pd.DataFrame(ch)
df

In [ ]:
reload(gd)

%matplotlib inline

L_c, areas, masses, AM = gd.fragmentation(1000, 10, 10, True, debris_category.rb, False)
N_fragments_total = L_c.shape[0]
lambda_c = np.log10(L_c)

def create_log_bins(values, nbins=100):
    #return np.geomspace(values.min(), values.max(), nbins)
    bins = np.geomspace(values.min(), values.max(), nbins)
    a = bins[1]/bins[0]
    bins = np.concatenate([[bins[0]/a], bins,[bins[-1]*a]])
    return bins

# # # Validating L_c
# h, b = np.histogram(L_c, bins=create_log_bins(L_c))
# plt.figure(figsize=(5,5))
# plt.xscale('log')
# plt.xlabel(r'$L_{c}$')
# plt.ylabel(r'$N_f$')
# plt.gca().set_xticks([1e-3, 1e-2, 1e-1, 1, 10])
# plt.gca().set_xlim([1e-3,10])
# plt.ticklabel_format(axis="y", style="sci", scilimits=(0,5))
# plt.gca().set_yticks([0, 2e5, 4e5, 6e5, 8e5])
# plt.gca().set_ylim([0,8e5])
# plt.plot((b[:-1] + b[1:])/2, h, '.-')
# plt.show()

# # # Validating Areas
# h, b = np.histogram(areas, bins=create_log_bins(areas))
# plt.figure(figsize=(5,5))
# plt.xscale('log')
# plt.xlabel(r'$Areas$')
# plt.ylabel(r'$N_f$')
# plt.gca().set_xticks([1e-8,1e-6, 1e-4, 1e-2, 1, 1e2])
# plt.gca().set_xlim([1e-8,1e2])
# plt.ticklabel_format(axis="y", style="sci", scilimits=(0,5))
# plt.gca().set_yticks([0, 2e5, 4e5, 6e5, 8e5])
# plt.gca().set_ylim([0,8e5])
# plt.plot((b[:-1] + b[1:])/2, h, '.-')
# plt.show()

# # Validating Mass & Velocity
h, b = np.histogram(masses, bins=create_log_bins(masses))
plt.figure(figsize=(5,5))
plt.xscale('log')
plt.xlabel(r'$Masses$')
plt.ylabel(r'$N_f$')
plt.gca().set_xticks([1e-8, 1e-5, 1e-2, 10, 1e4])
plt.gca().set_xlim([1e-8,1e4])
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,5))
plt.gca().set_yticks([0, 2e5, 4e5, 6e5, 8e5])
plt.gca().set_ylim([0,8e5])
plt.plot((b[:-1] + b[1:])/2, h, '.-')
plt.show()

# # # Validating Velocity
deltaV = np.array(gd.distribution_deltaV(AM, 10, False))
h, b = np.histogram(deltaV, bins=create_log_bins(deltaV))
plt.figure(figsize=(5,5))
plt.xscale('log')
plt.xlabel(r'$\delta V$')
plt.ylabel(r'$N_f$')
plt.gca().set_xticks([1e-3, 1e-2, 1e-1, 1, 10])
plt.gca().set_xlim([1e-3,10])
plt.grid()
plt.plot((b[:-1] + b[1:])/2, h, '.-')
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,5))
plt.gca().set_yticks([0, 2e5, 4e5, 6e5, 8e5])
plt.gca().set_ylim([0,8e5])
plt.show()

In [ ]:
deltaV 

<h3>2.3 and Onward will be alternate breakup models</h3>

<h1>3. Numerical Propagation</h1>

<h3>3.2 Calculating Orbital Debris Properties</h3>

In [5]:
from skyfield.api import load, EarthSatellite
reload(gd)

ts = load.timescale(builtin=True)

t_fragmentation = ts.now()
geocentric      = starlink.at(t_fragmentation)
init_position   = geocentric.position.m

m_target         = 250   # [kg] (Approx. mass of starlink sat)
m_projectile     = 200     # [kg]
v_impact         = 2   # [km·s^-1] (Measured relative to the target) (Needs to be in km·s^-1)
is_catastrophic  = False
is_explosion     = False

L_c, areas, masses, AM = gd.fragmentation(m_target, m_projectile, v_impact, is_catastrophic, debris_category.sc, is_explosion)
#deltaV = np.array(gd.distribution_deltaV(AM, v_impact, False)) # Returns as [km·s^-1]
#deltaV = deltaV * 1e3    #[m·s^-1]

/Users/reecehumphreys/Developer/Thesis/Thesis/generate_debris.py:368: RuntimeWarning: invalid value encountered in power
  found_L_c = np.sort((area / 0.556945)**(1/2.0047077)) # Inversing the Area function defined above


In [ ]:
np.max(deltaV /1e3)

In [ ]:
np.max(deltaV[deltaV/1e3 != 2.6] /1e3)

In [ ]:
I = deltaV/1e3 == 2.6
len(deltaV[I]) / len(deltaV) * 100


<h3>3.3 Converting Cartesian to Keplerian</h3>

In [ ]:
from numpy.linalg import norm
init_position = geocentric.position.m

reload(planetary_data)
reload(ct)

deb_positions       = np.empty((len(AM), 3))
deb_positions[:, :] = init_position[None,:]
deb_velocities      = gd.velocity_vectors(len(AM), geocentric.velocity.m_per_s, deltaV)

keplerian_state     = ct.rv2coe(deb_positions, deb_velocities, planetary_data.earth['mu'])

print(keplerian_state.shape)
# Removing fragments that would renter earth
periapsis           = keplerian_state[0, :] * (1 - keplerian_state[1, :])
I                   = np.argwhere(periapsis > planetary_data.earth['radius'])
keplerian_state     = np.squeeze(keplerian_state[:, I])
areas  = areas[I].flatten()  # When doing the indexing , a 1d dim being added which is unneccesary
masses              = masses[I].flatten() # When doing the indexing , a 1d dim being added which is unneccesary

In [ ]:
rand_index = 1

print("Cartesian")
print("r: ", deb_positions[rand_index, :])
print("v: ", deb_velocities[rand_index, :])
print("m: ", masses[rand_index])
print("m_E: ", planetary_data.earth['mass'])
print("Keplerian")
ks_rand = keplerian_state[:, rand_index]
print("a: ", ks_rand[0]) # a is tyically off in the thousands place
print("e: ", ks_rand[1])
print("i: ", ks_rand[2])
print("Omega: ", ks_rand[3]) 
print("omega: ", ks_rand[4])
print("M: ", ks_rand[5])
print("nu: ", ks_rand[6])
print()

<h3>3.4 Testing coverting back to Cartesian</h3>

In [ ]:
reload(ct)

%matplotlib notebook

indexes = np.random.default_rng().choice(keplerian_state.shape[1], size=1000, replace=False)
ks_sample = keplerian_state[:, indexes]

cartesian_state = np.array(ct.coe2rv_many(k=planetary_data.earth['mu'],
                              p=ks_sample[7, :],
                              ecc=ks_sample[1, :],
                              inc=ks_sample[2, :],
                              raan=ks_sample[3, :],
                              argp=ks_sample[4, :],
                              nu=ks_sample[6, :]))
radius_earth = planetary_data.earth['radius']
r = cartesian_state[0, :, :]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(r[:, 0], r[:, 1], r[:, 2])
ax.set_xlim3d([-radius_earth-1000*1e3, radius_earth+1000*1e3])
ax.set_xlabel('X (m)')

ax.set_ylim3d([-radius_earth-1000*1e3, radius_earth+1000*1e3])
ax.set_ylabel('Y (m)')

ax.set_zlim3d([-radius_earth-1000*1e3, radius_earth+1000*1e3])
ax.set_zlabel('Z (m)')

_u, _v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
_x = radius_earth*np.cos(_u)*np.sin(_v)
_y = radius_earth*np.sin(_u)*np.sin(_v)
_z = radius_earth*np.cos(_v)

ax.plot_surface(_x, _y, _z, cmap=cm.coolwarm)
plt.show()

In [ ]:
keplerian_state.shape
masses.shape

<h3>3.4 Debris Cloud formation</h3>

In [ ]:
## Shrink number of debris being used
indexes = np.random.default_rng().choice(keplerian_state.shape[1], size=250, replace=False)

ks = keplerian_state[:, indexes]
masses = masses[indexes]
areas = areas[indexes]
deb_positions = deb_positions[indexes, :]
deb_velocities = deb_velocities[indexes, :]

print(masses.shape)

In [ ]:
np.savez("debris.npz",
         keplerian_states=ks,
         masses=masses,
         areas=areas,
         deb_positions=deb_positions,
         deb_velocities=deb_velocities)

In [6]:
container = np.load('debris.npz')
ks = container['keplerian_states']
masses = container['masses']
areas = container['areas']
deb_positions = container['deb_positions']
deb_velocities = container['deb_velocities']

<h4>3.4.1 Ellisoid formation</h4>

In [157]:
import Perturbations as OP
import planetary_data as pd
from Perturbations import null_perts

reload(OP)

# Cleanup states to remove any fragments that would deorbit, given no perturbations
periapsis     = ks[0, :] * (1 - ks[1, :])
I             = np.argwhere(periapsis > pd.earth['radius'])
ks_pruned     = np.squeeze(ks[:, I])
T             = ks_pruned[8, :]
areas_pruned  = areas[I].flatten()  # When doing the indexing , a 1d dim being added which is unneccesary
masses_pruned = masses[I].flatten() # When doing the indexing , a 1d dim being added which is unneccesary

# Propagate orbit for a period of time
perts = null_perts()
perts['aero'] = True
perts['J2']   = False
op = OP.OrbitPropagator(ks_pruned, areas_pruned, masses_pruned, [0, 20*np.ceil(max(T))], 60*5, perts=perts)
op.propagate_orbit()

# Get the cartesian state representation
cartesian_states = op.cartesian_representation()


<h4>3.4.2 Particle Debris flux</h4>

Using a particle flux to determine when the fragments of the debris have finished the formation of the ring. Indicating the end of the first phase of the debris cloud formation. This is accomplished by creating an xz plane and detecting when particles have switched from one side to the other. This approach will cause a peak as fragments pass through that becomes uniform as the debris becomes uniformly spread out.

In [158]:
import time
import matplotlib.dates as mdates
from dateutil import tz

def fragmentation_flux(X):
    return np.sum((X[:-1, :, 1] < 0) & (X[1:, :, 1] > 0), axis=1)
    
position = cartesian_states[:, 0, :, :]
flux = fragmentation_flux(position)

<h4>3.4.3 Convergence of the flux</h4>

The next step is determining when the fragments have ended the torroid formation phase. This occurs when the fragments are approximately uniformally spread out. We can check to see when the flux meets a convergence criterion to determine when this happens.

In [181]:
w = 100 # Window of points to look at
tol = 1.5
convergence_ratio = np.array([np.var(flux[i:i+w])/np.mean(flux[i:i+w]) for i in range(len(flux))])    
intersection_index = np.argwhere(convergence_ratio <= tol).flatten()[0] # Grabbing the first point that meets the criteria

In [184]:
import datetime

# datetimes
t_flux = t_fragmentation.utc_datetime() + np.array(range(len(flux))) * datetime.timedelta(minutes = 5)

# Creating Flux v. Time plot
# fig = plt.figure()
# ax = fig.add_subplot(111)
# plt.plot(mdates.date2num(t_flux), flux, label='Flux')
# plt.axvline(x=t_flux[intersection_index], color='r',ls=':', label='Intersection')
# plt.xlim([t_flux[0], t_flux[-1]])
# formatter = mdates.DateFormatter('%m/%d %H:%M UTC', tz=tz.UTC)
# ax.xaxis.set_major_formatter(formatter)
# plt.legend(frameon=False)
# plt.xticks(rotation=60)
# plt.xlabel("Date-Time")
# plt.ylabel("Flux")
# plt.tight_layout()
# plt.show()

#Creating Convergence Ratio v. Time plot
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(mdates.date2num(t_flux), convergence_ratio, color='orange', label='Convergence ratio')
plt.axhline(y=tol, color='k', ls=":", label='Threshold')
plt.axvline(x=t_flux[intersection_index], color='r',ls=':', label='Intersection')
formatter = mdates.DateFormatter('%m/%d %H:%M UTC', tz=tz.UTC)
ax.xaxis.set_major_formatter(formatter)
plt.legend(frameon=False)
plt.xticks(rotation=60)
plt.xlim([t_flux[0], t_flux[-1]])
plt.yscale('symlog')
plt.xlabel("Date-Time")
plt.ylabel("Convergence ratio")
plt.tight_layout()
plt.show()

<h4>3.4.4 End of band formation</h4>

Now that the band has formed, we can shift away from propagating the exact position of each fragments and inplace propgate their changes in eccentricity and semi major axis due to drag. To do this first we must get the final states of the debris after the band has formed.

In [112]:
# Subsetting data such that just covers the formation of the band, i.e has uniformally spread out.
# cs_band = band cartesian state
# ks_band = band keplerian state
#from vpython import *

reload(vis)

cs_toroid = cartesian_states[:intersection_index, :, :, :]
ks_toroid = op.states[0:intersection_index, :, :]
op.states = ks_toroid

# Creating visual

# data = np.swapaxes(cs_toroid,0,1)[0, :, :, :]
# canvas()
# vis.visualize(data, op.cb)
#vis.generate_visualization(data, "Band_Formation", op.cb)
#op.states.shape

<h3> 4.0.0 Implementing Perturbations </h3>

In [178]:
op.tspan[-1] = 60 * 60 * 24 * 365 / 2  #[s]
op.dt = 60 * 60 * 1                   #[s]
de, da, domega, dOmega = op.propagate_perturbations()

Initializing perturbations with the following effects:
Aerodynamic perturbations ...
0.0
z0:  476266.8651176188
-0.028753280322373998
44614.49644842709
z0:  474926.9329785947
-0.029390818197887622
0.43074682394538827
z0:  476266.8521807585
-0.028753286410469668
0.6461202359180823
z0:  476266.84571232554
-0.02875328945451931
1.722987295781553
z0:  476266.8133701561
-0.028753304674774535
1.91443032864617
z0:  476266.80762043595
-0.02875330738059908
2.1537341197269413
z0:  476266.8004332837
-0.028753310762881075
2.1537341197269413
z0:  476266.8004332837
-0.028753310762881075
6.461202359180824
z0:  476266.6710645482
-0.02875337164402259
8.614936478907765
z0:  476266.60637988616
-0.028753402084780163
19.38360707754247
z0:  476266.2829562472
-0.028753554289206236
21.29803740618864
z0:  476266.2254585391
-0.028753581347936295
23.691075316996354
z0:  476266.1535863327
-0.028753615171418234
23.691075316996354
z0:  476266.1535863327
-0.028753615171418234
66.76575771153517
z0:  476264.8598858798


343815.1809830051
z0:  465001.06841540616
-0.034575204831839304
343815.1809830051
z0:  465001.0685563972
-0.03457520475205486
344526.6262109662
z0:  464975.5628429968
-0.03458964101659229
344882.34882494673
z0:  464962.79844118375
-0.03459686794624274
346660.9618948495
z0:  464898.9634920098
-0.03463303258393587
346977.1597739433
z0:  464887.6082497081
-0.03463946966832729
347372.4071228106
z0:  464873.4114482701
-0.03464751926487451
347372.4071228106
z0:  464873.4115611492
-0.03464751920086454
344284.1666738276
z0:  464984.2551543964
-0.03458472048094717
344518.6595192389
z0:  464975.8434368586
-0.03458948216734335
345691.1237462953
z0:  464933.7792431051
-0.03461330361057286
345899.56183110527
z0:  464926.2982201334
-0.034617541918991704
346160.1094371178
z0:  464916.9457320208
-0.034622841219615556
346160.1094371178
z0:  464916.9457643721
-0.034622841201283255
344026.3824382139
z0:  464993.49686489534
-0.034579489709661626
344131.9831658184
z0:  464989.71000185143
-0.034581632963369

460691.49417548627
z0:  460665.9853976909
-0.03711747339743954
461650.61376427783
z0:  460629.15342865884
-0.03713985519504896
461650.61376427783
z0:  460629.1553842407
-0.03713985400633628
463505.73045591224
z0:  460557.89441877883
-0.03718319570001667
464433.28880172945
z0:  460522.1735520577
-0.03720494054630718
469071.08053081547
z0:  460343.46633216925
-0.037313918498824795
469895.57683820854
z0:  460311.6415779693
-0.037333359098180786
470926.1972224499
z0:  460271.8405484669
-0.037357686372203575
470926.1972224499
z0:  460271.84301309753
-0.03735768486527591
463251.7532575659
z0:  460567.65049527865
-0.037177258961405585
464052.32300420996
z0:  460536.830726278
-0.037196016548477116
468055.1717374302
z0:  460382.6554952711
-0.03728999315367054
468766.78929000266
z0:  460355.2061137082
-0.037306749644061395
469656.3112307183
z0:  460320.8791980073
-0.03732771512484285
469656.3112307183
z0:  460320.88078354765
-0.037327714156187394
461970.84166293545
z0:  460616.8544064481
-0.0371

-0.03829658899173502
509718.431734268
z0:  458755.2164300401
-0.038296586764923805
500062.5700634289
z0:  459136.1432095608
-0.03805856936640703
500352.0362598013
z0:  459124.75302770175
-0.038065664880755794
501799.3672416632
z0:  459067.7917876858
-0.03810116874172977
502056.6705273276
z0:  459057.6599135287
-0.03810748739258872
502378.299634408
z0:  459044.99286586046
-0.038115388555211
502378.299634408
z0:  459044.9929465102
-0.03811538850489994
502957.2320271527
z0:  459022.1899637915
-0.03812961615937513
503246.69822352513
z0:  459010.77921301033
-0.03813673775745314
504694.02920538705
z0:  458953.7150902664
-0.038172372181869016
504951.3324910514
z0:  458943.56490576454
-0.0381787140819155
505272.9615981318
z0:  458930.8749621976
-0.03818664432166809
505272.9615981318
z0:  458930.8750432916
-0.03818664427098531
506231.460551324
z0:  458893.0534501029
-0.03821028963263866
506710.7100279201
z0:  458874.11717582773
-0.038222133741218044
509106.95741090056
z0:  458779.4071280062
-0.

-0.041192828106075816
618411.1300718092
z0:  454300.82698634267
-0.04119282805513671
618935.8969461999
z0:  454278.54372618534
-0.04120785388425891
619198.2803833954
z0:  454267.39323181007
-0.04121537483343253
620510.1975693725
z0:  454211.63083523046
-0.041253006874380764
620743.4272913239
z0:  454201.71230271086
-0.04125970413450205
621034.9644437633
z0:  454189.3120170422
-0.04126807867049202
621034.9644437633
z0:  454189.31209300645
-0.04126807861918435
618607.3372801192
z0:  454292.4954081103
-0.04119844548175276
618705.4408842742
z0:  454288.32837993465
-0.041201255317837915
619195.9589050493
z0:  454267.4918583743
-0.04121530830432293
619283.1621087426
z0:  454263.7868639752
-0.04121780760311223
619392.1661133593
z0:  454259.1553186299
-0.04122093214369643
619392.1661133593
z0:  454259.15532260295
-0.041220932141016055
619588.3733216693
z0:  454250.8182341056
-0.041226557117997976
619686.4769258243
z0:  454246.64844913036
-0.04122937073153745
620176.9949465994
z0:  454225.79814

...

nan
3076459.261785965
z0:  -6378100.0
nan
3076459.261785965
z0:  -6378100.0
nan
3078827.5718286885
z0:  -6378100.0
nan
3080011.72685005
z0:  -6378100.0
nan
3085932.5019568587
z0:  -6378100.0
nan
3086985.0841980693
z0:  -6378100.0
nan
3088300.811999582
z0:  -6378100.0
nan
3088300.811999582
z0:  -6378100.0
nan
3093848.784511037
z0:  -6378100.0
nan
3096622.770766765
z0:  -6378100.0
nan
3110492.702045402
z0:  -6378100.0
nan
3112958.4676060486
z0:  -6378100.0
nan
3116040.674556857
z0:  -6378100.0
nan
3116040.674556857
z0:  -6378100.0
nan
3121245.383302665
z0:  -6378100.0
nan
3123847.7376755686
z0:  -6378100.0
nan
3136859.509540088
z0:  -6378100.0
nan
3139172.713427114
z0:  -6378100.0
nan
3142064.218285896
z0:  -6378100.0
nan
3142064.218285896
z0:  -6378100.0
nan
3117715.934936826
z0:  -6378100.0
nan
3118553.565126811
z0:  -6378100.0
nan
3122741.7160767345
z0:  -6378100.0
nan
3123486.2762456094
z0:  -6378100.0
nan
3124416.9764567036
z0:  -6378100.0
nan
3124416.9764567036
z0:  -6378100.0
nan


z0:  -6378100.0
nan
3688858.772036513
z0:  -6378100.0
nan
3693142.155097861
z0:  -6378100.0
nan
3714559.070404602
z0:  -6378100.0
nan
3718366.5220146896
z0:  -6378100.0
nan
3723125.8365272987
z0:  -6378100.0
nan
3723125.8365272987
z0:  -6378100.0
nan
3682005.3591383556
z0:  -6378100.0
nan
3682862.035750625
z0:  -6378100.0
nan
3687145.4188119737
z0:  -6378100.0
nan
3687906.909133991
z0:  -6378100.0
nan
3688858.772036513
z0:  -6378100.0
nan
3688858.772036513
z0:  -6378100.0
nan
3690572.125261052
z0:  -6378100.0
nan
3691428.8018733216
z0:  -6378100.0
nan
3695712.18493467
z0:  -6378100.0
nan
3696473.6752566877
z0:  -6378100.0
nan
3697425.5381592093
z0:  -6378100.0
nan
3697425.5381592093
z0:  -6378100.0
nan
3700648.461047178
z0:  -6378100.0
nan
3702259.922491163
z0:  -6378100.0
nan
3710317.2297110856
z0:  -6378100.0
nan
3711749.639883516
z0:  -6378100.0
nan
3713540.1525990544
z0:  -6378100.0
nan
3713540.1525990544
z0:  -6378100.0
nan
3698497.537505703
z0:  -6378100.0
nan
3699033.53717895
z0

4144494.0889868396
z0:  -6378100.0
nan
4146394.2718664887
z0:  -6378100.0
nan
4146394.2718664887
z0:  -6378100.0
nan
4149814.601049857
z0:  -6378100.0
nan
4151524.765641541
z0:  -6378100.0
nan
4160075.5885999617
z0:  -6378100.0
nan
4161595.734903681
z0:  -6378100.0
nan
4163495.91778333
z0:  -6378100.0
nan
4163495.91778333
z0:  -6378100.0
nan
4167092.5613199146
z0:  -6378100.0
nan
4168890.883088207
z0:  -6378100.0
nan
4177882.4919296685
z0:  -6378100.0
nan
4179481.0001681503
z0:  -6378100.0
nan
4181479.135466253
z0:  -6378100.0
nan
4181479.135466253
z0:  -6378100.0
nan
4166395.5952622257
z0:  -6378100.0
nan
4167845.4340016735
z0:  -6378100.0
nan
4175094.6276989123
z0:  -6378100.0
nan
4176383.373245088
z0:  -6378100.0
nan
4177994.305177808
z0:  -6378100.0
nan
4177994.305177808
z0:  -6378100.0
nan
4180893.9826567033
z0:  -6378100.0
nan
4182343.821396151
z0:  -6378100.0
nan
4189593.01509339
z0:  -6378100.0
nan
4190881.7606395655
z0:  -6378100.0
nan
4192492.6925722854
z0:  -6378100.0
nan
41

4867406.972795858
z0:  -6378100.0
nan
4876926.0478186095
z0:  -6378100.0
nan
4924521.422932363
z0:  -6378100.0
nan
4932982.822952585
z0:  -6378100.0
nan
4943559.572977864
z0:  -6378100.0
nan
4943559.572977864
z0:  -6378100.0
nan
4983769.768998089
z0:  -6378100.0
nan
5003874.867008203
z0:  -6378100.0
nan
5104400.357058766
z0:  -6378100.0
nan
5122271.555289977
z0:  -6378100.0
nan
5144610.553078991
z0:  -6378100.0
nan
5144610.553078991
z0:  -6378100.0
nan
5182253.712826724
z0:  -6378100.0
nan
5201075.2927005915
z0:  -6378100.0
nan
5295183.192069924
z0:  -6378100.0
nan
5311913.485291139
z0:  -6378100.0
nan
5332826.351817657
z0:  -6378100.0
nan
5332826.351817657
z0:  -6378100.0
nan
5152139.185028538
z0:  -6378100.0
nan
5155903.501003311
z0:  -6378100.0
nan
5174725.080877177
z0:  -6378100.0
nan
5178071.139521421
z0:  -6378100.0
nan
5182253.712826724
z0:  -6378100.0
nan
5182253.712826724
z0:  -6378100.0
nan
5151218.963747959
z0:  -6378100.0
nan
5154523.169082442
z0:  -6378100.0
nan
5171044.19

z0:  -6378100.0
nan
7308187.064291065
z0:  -6378100.0
nan
7372821.029010604
z0:  -6378100.0
nan
7405138.011370375
z0:  -6378100.0
nan
7566722.9231692245
z0:  -6378100.0
nan
7595449.129711241
z0:  -6378100.0
nan
7631356.887888764
z0:  -6378100.0
nan
7631356.887888764
z0:  -6378100.0
nan
7746393.682365847
z0:  -6378100.0
nan
7803912.079604388
z0:  -6378100.0
nan
8091504.065797094
z0:  -6378100.0
nan
8142631.530009131
z0:  -6378100.0
nan
8206540.860274177
z0:  -6378100.0
nan
8206540.860274177
z0:  -6378100.0
nan
9356908.805045003
z0:  -6378100.0
nan
9932092.777430415
z0:  -6378100.0
nan
12808012.639357481
z0:  -6378100.0
nan
13319287.281477848
z0:  -6378100.0
nan
13958380.584128307
z0:  -6378100.0
nan
13958380.584128307
z0:  -6378100.0
nan
14319584.467302646
z0:  -6378100.0
nan
14500186.408889815
z0:  -6378100.0
nan
15403196.116825663
z0:  -6378100.0
nan
15563731.176014256
z0:  -6378100.0
nan
15764400.0
z0:  -6378100.0
nan
15764400.0
z0:  -6378100.0
nan


(250, 365)

In [180]:
perigee = da * (1 - de)
altitude = perigee - op.cb['radius']

frag1 =  perigee[1, :]
frag1 = np.trim_zeros(frag1)
plt.plot(frag1 / 1e3)
plt.show()

In [83]:
da_0 = da[:, 0]
I = np.random.default_rng().choice(len(da_0), size=5, replace=False)

In [ ]:
print(times / (60 * 60 * 24), ' days')

In [104]:
periapsis = da * (1 - de)

In [107]:
da[0, :] / 1e3

array([6854.36686512, 6854.36686512, 6854.36686511, 6854.36686511,
       6854.36686511, 6854.3668651 , 6854.3668651 , 6854.3668651 ,
       6854.3668651 , 6854.36686509, 6854.36686509, 6854.36686509,
       6854.36686509, 6854.36686508, 6854.36686508, 6854.36686508,
       6854.36686508, 6854.36686507, 6854.36686507, 6854.36686507,
       6854.36686507, 6854.36686506, 6854.36686506, 6854.36686506,
       6854.36686506, 6854.36686505, 6854.36686505, 6854.36686505,
       6854.36686504, 6854.36686504, 6854.36686504, 6854.36686504,
       6854.36686503, 6854.36686503, 6854.36686503, 6854.36686503,
       6854.36686502, 6854.36686502, 6854.36686502, 6854.36686502,
       6854.36686501, 6854.36686501, 6854.36686501, 6854.36686501,
       6854.366865  , 6854.366865  , 6854.366865  , 6854.366865  ,
       6854.36686499, 6854.36686499, 6854.36686499, 6854.36686499,
       6854.36686498, 6854.36686498, 6854.36686498, 6854.36686497,
       6854.36686497, 6854.36686497, 6854.36686497, 6854.36686